# Задача 1 - триграмная языковая модель

далее идет повторение части кода из тетрадки 

In [1]:
from tqdm.auto import tqdm

In [2]:
def get_current_memory_usage():
    ''' Memory usage in gB '''

    with open('/proc/self/status') as f:
        memusage = f.read().split('VmRSS:')[1].split('\n')[0][:-3]

    return f"{int(memusage.strip()) / 1024.0 / 1024.0:.4f} GB"

In [3]:
get_current_memory_usage()

'0.0470 GB'

In [4]:
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные
dvach = open('data/2ch_corpus.txt').read()
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные

news = open('data/lenta.txt').read()

По длине оно сопоставимы.

In [5]:
print("Длина 1 -", len(dvach))
print("Длина 2 -", len(news))

Длина 1 - 11638405
Длина 2 - 11536552


Напишем простую функцию для нормализации. 

In [6]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


Сравним тексты по токенам

In [7]:
norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [8]:
print("Длина корпуса токсичных постов в токенах -", len(norm_dvach))
print("Длина корпуса новостных текстов в токенах - ", len(norm_news))

Длина корпуса токсичных постов в токенах - 1858941
Длина корпуса новостных текстов в токенах -  1505789


И по уникальным токенам

In [9]:
print("Уникальных токенов в корпусе токсичных постов -", len(set(norm_dvach)))
print("Уникальный токенов в корпусе новостных текстов - ", len(set(norm_news)))

Уникальных токенов в корпусе токсичных постов - 189515
Уникальный токенов в корпусе новостных текстов -  116302


In [10]:
from collections import Counter

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [13]:
sentences_dvach = [['<start>'] + normalize(text) + ['<end>'] for text in tqdm(sent_tokenize(dvach))]
sentences_news = [['<start>'] + normalize(text) + ['<end>'] for text in tqdm(sent_tokenize(news))]

## А вот тут помимо bigrams_* создаем и trigrams_*

In [14]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in tqdm(sentences_dvach):
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, n=3))


unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in tqdm(sentences_news):
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))


In [15]:
trigrams_news.most_common(10)

[('<start> об этом', 1579),
 ('<start> по словам', 1549),
 ('сообщает риа новости', 1324),
 ('со ссылкой на', 1243),
 ('риа новости <end>', 1228),
 ('<start> кроме того', 1070),
 ('<start> как сообщает', 1064),
 ('<start> напомним что', 1006),
 ('по его словам', 899),
 ('<start> по его', 868)]

Тут пробуем считать вероятность по триграммам, нормируя на количество вхождений биграм

In [16]:
# phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала''Ныть надо меньше и работать больше.'
phrase = 'как вы смотрите эту залупу, серьезно. в чем прикол ваще это смотреть?'
prob = Counter()
for ngram in ngrammer(['<start>'] + normalize(phrase) + ['<end>'], n=3):
    word1, word2, word3 = ngram.split()
    
    if ' '.join([word1, word2]) in bigrams_dvach and ngram in trigrams_dvach:
        prob['dvach'] += np.log(trigrams_dvach[ngram]/bigrams_dvach[' '.join([word1, word2])])
    else:
        prob['dvach'] += np.log(0.001)
    
    if ' '.join([word1, word2]) in bigrams_news and ngram in trigrams_news:
        prob['news'] += np.log(trigrams_news[ngram]/bigrams_news[' '.join([word1, word2])])
    else:
        prob['news'] += np.log(0.001)

prob['news'] = np.exp(prob['news'])
prob['dvach'] = np.exp(prob['dvach'])

Работает получше, чем вариант с биграммами, вероятности отличаются на 15 порядков

In [17]:
prob.most_common()

[('dvach', 1.6929191136207173e-14), ('news', 1.0000000000000028e-36)]

### Собираем матрицу вероятностей

умещаем вероятности в одну матрицу - сначала идут униграммы, дальше биграммы(хотя по идее униграммы тут уже и не нужны)

apparently пришлось использовать fp16 для матрицы двача, т.к иначе не хватало даже имеющегося терабайта RAM

In [18]:
matrix_dvach = np.zeros(
    (len(unigrams_dvach)+len(bigrams_dvach), len(unigrams_dvach)), dtype=np.float16
)
print(get_current_memory_usage())
id2word_dvach = list(unigrams_dvach)+list(bigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

# заполняем вероятности биграммной части
for ngram in tqdm(bigrams_dvach):
    word1, word2 = ngram.split()
    matrix_dvach[word2id_dvach[word1]][word2id_dvach[word2]] =  (bigrams_dvach[ngram]/
                                                                     unigrams_dvach[word1])
# заполняем вероятности триграммной части
for ngram in tqdm(trigrams_dvach):
    word1, word2, word3 = ngram.split()
    bigram = " ".join([word1, word2])
    matrix_dvach[word2id_dvach[bigram]][word2id_dvach[word3]] = trigrams_dvach[ngram]/bigrams_dvach[bigram]

1.5966 GB


In [19]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:          1.0Ti        76Gi       388Gi       2.0Mi       542Gi       925Gi
Swap:         8.0Gi       9.0Mi       8.0Gi


In [21]:
matrix_news = np.zeros(
    (len(unigrams_news)+len(bigrams_news), len(unigrams_news)), dtype=np.float16
)
print(get_current_memory_usage())

id2word_news = list(unigrams_news)+list(bigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

# заполняем вероятности биграммной части
for ngram in tqdm(bigrams_news):
    word1, word2 = ngram.split()
    matrix_news[word2id_news[word1]][word2id_news[word2]] =  (bigrams_news[ngram]/
                                                                     unigrams_news[word1])
# заполняем вероятности триграммной части
for ngram in tqdm(trigrams_news):
    word1, word2, word3 = ngram.split()
    bigram = " ".join([word1, word2])
    matrix_news[word2id_news[bigram]][word2id_news[word3]] = trigrams_news[ngram]/bigrams_news[bigram]

8.2325 GB


In [22]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:          1.0Ti        82Gi       382Gi       2.0Mi       542Gi       919Gi
Swap:         8.0Gi       9.0Mi       8.0Gi


## Генерация по триграммам

In [23]:
def generate_trigrams(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    next_word = id2word[np.random.choice(matrix.shape[1], p=matrix[current_idx])] 
    ## генерируем второе слово
    ## чтобы дальше генерировать только по биграммам
    text += [next_word]
    current_idx = word2id[" ".join([start, next_word])] 
    ## current_idx = id сгенерированной биграммы вида '<start> word'
    
    for i in range(n):
        chosen_idx = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        prev_word = text[-1]
        text.append(id2word[chosen_idx])
        chosen = word2id[" ".join([prev_word, id2word[chosen_idx]])]  
        ## следующий id - id биграммы
        
        if id2word[chosen_idx] == '<end>': 
            ## если встретили <end> - повторяем процедуру из начала генерации
            chosen = word2id['<start>']
            next_word_idx = np.random.choice(matrix.shape[1], p=matrix[chosen])
            text.append(id2word[next_word_idx])
            chosen = word2id[" ".join(["<start>", id2word[next_word_idx]])]
            ## собираем биграмму из <start> и сгенерированного слова
            
        current_idx = chosen
    
    return ' '.join(text)

## Для сравнения - оригинальный вариант для генерации по биграммам

In [24]:
def generate_bigrams(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

### Сравним генерацию с триграммами и биграммами

In [25]:
np.random.seed(42)
print("="*50)
print("Trigram Model")
print("="*50)
print(generate_trigrams(matrix_dvach, id2word_dvach, word2id_dvach).replace('<end>', '\n'))

np.random.seed(42)
print("="*50)
print("Bigram Model")
print("="*50)
print(generate_bigrams(matrix_dvach, id2word_dvach, word2id_dvach).replace("<end>", "\n"))

Trigram Model
я сую тебе за щеку 
 по потерям хохлов это не револююция а русский язык для старта языки на которых танк ебет на 5 лет подрались даже не знают и умеют больше чем цис-тянки 
 свободный человек и без всяких своих субъективных оценочных суждений 
 the witch cult has been taken down until further notice 
 это уже можно было бы похуй если бы у тебя там пиздецома 
 б-ушные машины стоят копейки бэкдор через фб не работает-проверил вчера 
 химия объясняет как и я ещё тогда удивился физике падения тел уровень он обозревал тот что за тест 
 только с сегой а значит братушки хотят тебя выпилить
Bigram Model
я пруфану бицуху после одного я в мужском роде был 
 libreoffice либеральный анон 
 тебе как они потом когда будешь делать на хуец мой владыка с няса или на расстройство 
 на двачеокажется онанист однако он там русским который не тупой и ракетами-носителями 
 the health news джве школы пионерия мертва рррряяя наносим ответный огонь 
 у меня больше чем то святое 
 как рассказать о 

In [26]:
np.random.seed(42)
print("="*50)
print("Trigram Model")
print("="*50)
print(generate_trigrams(matrix_news, id2word_news, word2id_news).replace('<end>', '\n'))

np.random.seed(42)
print("="*50)
print("Bigram Model")
print("="*50)
print(generate_bigrams(matrix_news, id2word_news, word2id_news).replace('<end>', '\n'))

Trigram Model
из оружейного тайника где басаев хранил документы компрометирующие других главарей бандформирований перенести боевые действия 
 по словам иванова речь не идет 
 так не ведет войну в чечне владимир путин выступил перед представителями компаний boeing rocketdyne hughes и rockwell 
 в течение года вкладывает в развитие ряда отраслей экономики 
 sotheby s сталкивается сегодня с вынесением определения судебной коллегии по гражданским делам мосгорсуда решение межмуниципального суда москвы согласно которому юная уильямс должна продолжать выступать в кубке уефа 
 как заявил рбк заместитель начальника российского бюро интерпола алексея абрамова сотрудники мвд не готово заключить честную сделку в интересах следствия 
 в свою очередь отрицают какую-либо причастность к взрывам
Bigram Model
из контрабандной иракской нефти 
 по его политическом пространстве института 
 рнк-компьютер разработанный на своих акций был кошман заявил что покушение с собой гибель большой мощности 
 еще 2 
 п

## Заключение
--------------
Действительно, триграммная модель генерирует гораздо более связаный текст, чем модель на биграммах. Однако даже триграммная модель ещё далека от того, чтобы генрировать длинные адекватно-связанные тексты. Очевидно это будет улучшаться с добавлением следующих уровней n-граммности, однако тут встает вопрос с ограниченным объемом оперативной памяти.

P.S.: я решил не добавлять ещё один тег //<start//> в начало, вместо этого я догенерирую второй токен в начале предложения в униграммном 'режиме'